In [70]:
# Importing the necessary libraries for AI2-THOR to run

!pip install --upgrade ai2thor ai2thor-colab &> /dev/null
import ai2thor
import ai2thor_colab
import time
import math
from typing import Dict, List

from ai2thor.controller import Controller
from ai2thor_colab import (
    plot_frames,
    show_objects_table,
    side_by_side,
    overlay,
    show_video
)

# Class for controlling robot navigation. This is where we will have all the navigation commands.
# This has NOT yet got the LLM connected, but merely a set of tools to move the robot and to interact
# with the simulation environment.
class RobotNavigationControl:
    is_DEBUG = False

    # Starts server
    def start_ai2_thor(self):
        ai2thor_colab.start_xserver()
        "AI2-THOR Version: " + ai2thor.__version__

    # Initialises controller
    def initialise_controller(self):
        self.controller = Controller(
            agentMode="default",
            visibilityDistance=1.5,
            scene="FloorPlan209",
            # image modalities
            #renderDepthImage=False,
            #renderInstanceSegmentation=False,
            # camera properties
            width=400,
            height=400,
            fieldOfView=120,
            # step sizes
            gridSize=0.25,
            snapToGrid=True,
            #rotateStepDegrees=15,
        )

        # If debug is enabled, then print scene name and a few other things.
        if (self.is_DEBUG):
            event = self.controller.step(action="RotateRight")
            metadata = event.metadata
            print(event, event.metadata.keys())
            print("sceneName : " + self.controller.last_event.metadata["sceneName"])
            print("agent_pos : " + str(self.controller.last_event.metadata["agent"]["position"]))
            print("agent_rtn : " + str(self.controller.last_event.metadata["agent"]["rotation"]))
            #print("actionReturn : " + controller.last_event.metadata["actionReturn"])

    # Print a table of all objects in the scene
    def show_all_objects(self):
        show_objects_table(self.controller.last_event.metadata['objects'])
        print(self.controller.last_event.metadata['objects'])
    
    # Rotate left by given number of degrees degrees
    def rotate_left(self, deg):
        frames = []
        for _ in range(int(deg) // 5):
            frames.append(self.controller.step(action="RotateLeft", degrees=5).frame)
            time.sleep(0.05)
        
    # Rotate right by given number of degrees degrees
    def rotate_right(self, deg):
        frames = []
        for _ in range(int(deg) // 5):
            frames.append(self.controller.step(action="RotateRight", degrees=5).frame)
            time.sleep(0.05)
        
    # Rotate right or left depending on the degree (positive degree- right, negative - left)
    def rotate_by_degree(self, deg):
        if (deg < 0):
            self.rotate_left(abs(deg))
        else:
            self.rotate_right(abs(deg))
        
    # Store visible objects in the self.visible_objects collection and print them out if needed
    def get_visible_objects(self, print_objects = False):
        objects = self.controller.last_event.metadata['objects']
        visible_objects = []

        for obj in objects:
            if obj['visible']:
                if print_objects:
                    print(obj['objectType'] + " : " + str(obj['position']))
                visible_objects.append(obj)
                
        return visible_objects
        
    # Find the closest positon from the given reachable positions to the given object position using
    # Pythagorean theorem.
    def closest_position(self, object_position: Dict[str, float], reachable_positions: List[Dict[str, float]]) -> Dict[str, float]:
        out = reachable_positions[0]
        min_distance = float('inf')
        for pos in reachable_positions:
            # NOTE: y is the vertical direction, so only care about the x/z ground positions
            dist = sum([(pos[key] - object_position[key]) ** 2 for key in ["x", "z"]])
            if dist < min_distance:
                min_distance = dist
                out = pos
        return out

    # Finds the given object name in the given collection of objects and if found, returns the actual object
    def validate_object_in_collection(self, obj_name, obj_collection):        
        obj_names = sorted([obj["objectType"] for obj in obj_collection])
        
        try:
            assert obj_name in obj_names
        except AssertionError:
            print(obj_name + " is not visible!!!!!!!!!!!!!!!!")
            return None
        
        obj_of_interest = next(obj for obj in obj_collection if obj["objectType"] == obj_name)
        return obj_of_interest
    
    # Navigate to object defined by the name in the input
    def navigate_to_object(self, obj_name):
        #plot_frames(self.controller.last_event)
        obj_navigate_to = self.validate_object_in_collection(obj_name, self.get_visible_objects())
        
        # Can't navigate to an unknown object
        if (obj_navigate_to is None):
            return
        
        reachable_positions = self.controller.step(action="GetReachablePositions").metadata["actionReturn"]
        
        pos_navigate_to = self.closest_position(obj_navigate_to['position'], reachable_positions)
        
        #print(pos_navigate_to)
        
        self.controller.step(action="Teleport", **pos_navigate_to)
        self.controller.step(action="MoveBack")
        #plot_frames(self.controller.last_event)
        
    # Print pose of the object defined by the name in the input
    def print_pose_of_object(self, obj_name):
        obj_of_interest = self.validate_object_in_collection(obj_name, self.controller.last_event.metadata['objects'])
        
        # Unknown object
        if (obj_of_interest is None):
            return
        
        print("position of " + obj_name + " : " + str(obj_of_interest["position"]))
        print("rotation of " + obj_name + " : " + str(obj_of_interest["rotation"]))
        
    # Print current pose of robot
    def print_current_pose_of_robot(self):
        print("agent_pos : " + str(self.controller.last_event.metadata["agent"]["position"]))
        print("agent_rtn : " + str(self.controller.last_event.metadata["agent"]["rotation"]))
        
    def get_angle_offset_from_target(self, obj_name):
        obj_of_interest = self.validate_object_in_collection(obj_name, self.controller.last_event.metadata['objects'])
        
        # Unknown object
        if (obj_of_interest is None):
            return None
        
        robot_position = self.controller.last_event.metadata["agent"]["position"]
        
        # Using formula tg(alpha) = a/b to find relative angle from robot to object
        #tg_alpha = (robot_position["z"] - obj_of_interest["position"]["z"]) / (robot_position["x"] - obj_of_interest["position"]["x"])
        z_diff = (robot_position["z"] - obj_of_interest["position"]["z"])
        x_diff = (robot_position["x"] - obj_of_interest["position"]["x"])
        #print(math.degrees(math.atan2(z_diff, x_diff)))
        return math.degrees(math.atan2(z_diff, x_diff))
        
    def rotate_to_face_target(self, obj_name):
        angle_to_target = self.get_angle_offset_from_target(obj_name)
        
        if (angle_to_target is None):
            print(obj_name + " not visibile")
            return
        
        current_robot_yaw = self.controller.last_event.metadata["agent"]["rotation"]["y"]
        
        self.rotate_by_degree(angle_to_target - current_robot_yaw)
        print("rotating by: " + str(angle_to_target - current_robot_yaw))
        
        self.controller.step(action="MoveBack")

In [71]:
rnc = RobotNavigationControl()
rnc.start_ai2_thor()
rnc.initialise_controller()

rnc.rotate_left(180)

rnc.navigate_to_object("Vase")
rnc.print_current_pose_of_robot()
#rnc.get_angle_offset_from_target("Vase")

rnc.rotate_to_face_target("Vase")
rnc.print_current_pose_of_robot()

agent_pos : {'x': -2.5, 'y': 0.9027014970779419, 'z': -3.5}
agent_rtn : {'x': -0.0, 'y': 180.00001525878906, 'z': 0.0}


AttributeError: module 'math' has no attribute 'abs'